In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import datetime
from datetime import datetime
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, multilabel_confusion_matrix, confusion_matrix, precision_score, recall_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# from catboost import CatBoostClassifier
# from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Lasso
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [ ]:
df = pd.read_csv('dataset.csv')

In [ ]:
df.isna().sum()

encounter_id                       0
patient_id                         0
hospital_id                        0
age                             4228
bmi                             3429
                               ...  
solid_tumor_with_metastasis      715
apache_3j_bodysystem            1662
apache_2_bodysystem             1662
Unnamed: 83                    91713
hospital_death                     0
Length: 85, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91713 entries, 0 to 91712
Data columns (total 85 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   encounter_id                   91713 non-null  int64  
 1   patient_id                     91713 non-null  int64  
 2   hospital_id                    91713 non-null  int64  
 3   age                            87485 non-null  float64
 4   bmi                            88284 non-null  float64
 5   elective_surgery               91713 non-null  int64  
 6   ethnicity                      90318 non-null  object 
 7   gender                         91688 non-null  object 
 8   height                         90379 non-null  float64
 9   icu_admit_source               91601 non-null  object 
 10  icu_id                         91713 non-null  int64  
 11  icu_stay_type                  91713 non-null  object 
 12  icu_type                       91713 non-null 

In [ ]:
df.drop(["encounter_id" ,"patient_id" ,"hospital_id","Unnamed: 83", ],axis =1, inplace=True) # Not relevant to predict if a patient died or not

In [ ]:
# Dealing with numerical columns
Num=[]
for col in df.columns:
    if (df[col].dtype==int)or (df[col].dtype==float):
        Num.append(col)


for col in df.columns:
    if col in Num :
        df[col].fillna(df[col].median(), inplace=True)
        
        
# Dealing with categorical columns
categorical=[]
for col in df.columns:
    if  (df[col].dtype==object):
        categorical.append(col)

for col in df.columns:
    if col in categorical:
        df[col].fillna(df[col].mode(), inplace=True)

In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 KB 3.4 MB/s eta 0:00:00


Modelling Section:

In [ ]:
from category_encoders import CountEncoder
CE = CountEncoder(normalize=True, cols=categorical)
data = CE.fit_transform(df)

In [ ]:
from sklearn.preprocessing import RobustScaler
RS = RobustScaler()
scale = RS.fit_transform(data)
data = pd.DataFrame(scale, columns=data.columns)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

### Section 0: XG Boost, XG Boost with LDA & XG Boost with PCA ###

In [ ]:
# XGB 

model = xgb.XGBRegressor(seed = 20, objective = 'multi:softmax', 
           max_depth = 6,
           learning_rate = 0.1,
           subsample = 0.7,
           colsample_bytree = 0.2,
           colsample_bylevel = 0.5,
           n_estimators = 100, num_class = 2)

model.fit(X_train,y_train)
pred = model.predict(X_test)
print(confusion_matrix(y_test, pred))
print(f'Acuracy of the xgboost{accuracy_score(y_test, pred)}')
print(classification_report(y_test, pred))

In [ ]:
#XGB PCA

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

# Apply PCA with an explained_variance ratio of 0.7
pca = PCA(n_components = 0.7)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Create the XGBoost model
model = xgb.XGBRegressor(seed = 20, objective = 'multi:softmax', 
           max_depth = 6,
           learning_rate = 0.1,
           subsample = 0.7,
           colsample_bytree = 0.2,
           colsample_bylevel = 0.5,
           n_estimators = 100, num_class = 2)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
pred = model.predict(X_test)

# Evaluate the model's performance
print(confusion_matrix(y_test, pred))
print(f'Accuracy of the xgboost: {accuracy_score(y_test, pred)}')
print(classification_report(y_test, pred))


In [ ]:
# XGB LDA

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

# Apply LDA
lda = LDA()
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

# Create the XGBoost model
model = xgb.XGBRegressor(seed = 20, objective = 'multi:softmax', 
           max_depth = 6,
           learning_rate = 0.1,
           subsample = 0.6,
           colsample_bytree = 0.3,
           colsample_bylevel = 0.5,
           n_estimators = 100, num_class = 2)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data

pred = model.predict(X_test)

# Evaluate the model's performance
print(confusion_matrix(y_test, pred))
print(f'Accuracy of the xgboost: {accuracy_score(y_test, pred)}')
print(classification_report(y_test, pred))



In [ ]:
#BernoulliNB
from sklearn.naive_bayes import BernoulliNB
X_train, X_test, y_train, y_test = train_test_split(data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

# Initialize the model
bnb = BernoulliNB()

bnb.fit(X_train, y_train)

pred = bnb.predict(X_test)

# Evaluate the model's performance
print(confusion_matrix(y_test, pred))
print(f'Accuracy of the BernoulliNB: {accuracy_score(y_test, pred)}')
print(classification_report(y_test, pred))


In [ ]:
#BernoulliNB PCA

from sklearn.naive_bayes import BernoulliNB
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

# Apply PCA
pca = PCA(n_components=0.7)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Initialize the model
bnb = BernoulliNB()

# Fit the model to the data
bnb.fit(X_train, y_train)

# Make predictions
pred = bnb.predict(X_test)

# Evaluate the model's performance
print(confusion_matrix(y_test, pred))
print(f'Accuracy of the BernoulliNB: {accuracy_score(y_test, pred)}')
print(classification_report(y_test, pred))



In [ ]:
# BernoulliNB LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

# Apply LDA
lda = LDA(n_components=1)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

# Initialize the model
bnb = BernoulliNB(alpha = [1.0],
              fit_prior = [True],
              binarize = [1.0])

# Fit the model to the data
bnb.fit(X_train, y_train)

# Make predictions
pred = bnb.predict(X_test)

# Evaluate the model's performance
print(confusion_matrix(y_test, pred))
print(f'Accuracy of the BernoulliNB: {accuracy_score(y_test, pred)}')
print(classification_report(y_test, pred))


In [ ]:
# lightgbm 
from sklearn.model_selection import train_test_split
# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

# Initialize the model with the specified parameters
lgbm = LGBMClassifier(learning_rate=0.01, n_estimators=1000, max_depth=9)

# Fit the model to the training data
lgbm.fit(X_train, y_train)

# Make predictions on the test data
pred = lgbm.predict(X_test)


# Evaluate the model's performance
print(confusion_matrix(y_test, pred))
print(f'Accuracy of the LGBMClassifier: {accuracy_score(y_test, pred)}')
print(classification_report(y_test, pred))


In [ ]:
##lightgbm PCA
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from lightgbm import LGBMClassifier

X_train, X_test, y_train, y_test = train_test_split(data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

# Apply PCA
pca = PCA(n_components=0.7)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Initialize the model
lgbm = LGBMClassifier(learning_rate=0.01, n_estimators=1000, max_depth=9)

# Fit the model to the data
lgbm.fit(X_train, y_train)

# Make predictions
pred = lgbm.predict(X_test)

# Evaluate the model's performance
print(confusion_matrix(y_test, pred))
print(f'Accuracy of the LGBMClassifier: {accuracy_score(y_test, pred)}')
print(classification_report(y_test, pred))


In [ ]:
# lightgbm LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from lightgbm import LGBMClassifier

X_train, X_test, y_train, y_test = train_test_split(data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

# Apply LDA
lda = LDA(n_components=1)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

# Initialize the model
lgbm = LGBMClassifier(learning_rate=0.01, n_estimators=1000, max_depth=9)

# Fit the model to the data
lgbm.fit(X_train, y_train)

# Make predictions
pred = lgbm.predict(X_test)

# Evaluate the model's performance
print(confusion_matrix(y_test, pred))
print(f'Accuracy of the LGBMClassifier: {accuracy_score(y_test, pred)}')
print(classification_report(y_test, pred))


### Section 1: Naive Bayes, KNN, Linear SVM, and SVM###

In [ ]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB
classifier = BernoulliNB()
classifier.fit(X_train, y_train)
pred = classifier.predict(X_test)
print(f'Acuracy of the Gaussian Naive Bayes:{accuracy_score(y_test, pred)}')
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
# KNN
parameters = {'n_neighbors':list(range(20,31))}
neigh = KNeighborsClassifier()
clf = GridSearchCV(neigh, parameters)
clf.fit(X_train, y_train)
print(clf.best_params_)
pred = clf.predict(X_test)
print(f'Acuracy of the KNN{clf.best_params_}:{accuracy_score(y_test, pred)}')
print(classification_report(y_test, pred))

# 16756 patients didn't die vs 1587 patients died
# Out of all the players that the model predicted would die in hospital, only 75% actually did.
# Out of all the patients that actually did die in the hospital, the model only predicted this outcome 
# correctly for 19% of those players
# F1-score = 2 * (Precision * Recall) / (Precision + Recall)

# We are predicting accurately 0s but not ones

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=27)
neigh.fit(X_train, y_train)
pred = neigh.predict(X_test)
print(f'Acuracy of the KNN27:{accuracy_score(y_test, pred)}')
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

# Cannot calculate feature importance in KNN

In [ ]:
# Linear SVM
searcher = LinearSVC(penalty = 'l1', dual = False, max_iter = 10000)
parameters = {'C': [0.01, 0.03, 0.05, 0.055, 0.06, 0.065, 0.07, 0.075]}
searcher = GridSearchCV(searcher, parameters)
searcher.fit(X_train, y_train)
print(searcher.best_params_)
pred = searcher.predict(X_test)
print(f'Acuracy of the SVM{searcher.best_params_}:{accuracy_score(y_test, pred)}')
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
svc = LinearSVC(penalty = 'l1', dual = False, max_iter = 10000, C = 0.05)
svc.fit(X_train, y_train)
pred = svc.predict(X_test)
print(f'Acuracy of the SVM:{accuracy_score(y_test, pred)}')
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

# Calculate feature importance
# importances = pd.DataFrame(data={
#     'Attribute': X_train.columns,
#     'Importance': svc.coef_[0]
# })
# importances = importances.sort_values(by='Importance', ascending=False)
# plt.bar(x=importances['Attribute'], height=importances['Importance'], color='#087E8B')
# plt.title('Feature importances obtained from coefficients', size=20)
# plt.xticks(rotation='vertical')
# plt.show()

# I only want features with importance higher than 0.02
# importances = importances[importances['Importance']>0.02]

# X_train, X_test, y_train, y_test = train_test_split(
#     data[importances['Attribute'].tolist()], data['hospital_death'], test_size=0.2, random_state=42)
# This will not increase the recall of the model

In [ ]:
# SVM
# I tested all these kernels: linear, poly, rbf, sigmoid and the best one is sigmoid with C = 0.7
svc = SVC(max_iter = -1, kernel = 'sigmoid', C = 0.7)
svc.fit(X_train, y_train)
pred = svc.predict(X_test)
print(f'Acuracy of the SVM:{accuracy_score(y_test, pred)}')
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

### Section 2: KNN, Linear SVM and SVM with PCA ###

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

from sklearn.decomposition import PCA
pca = PCA(n_components=0.7) # I tried 0.5 and 0.8, worser results
X_train = pca.fit_transform(X_train, y_train)
X_test = pca.transform(X_test)

In [ ]:
# KNN
parameters = {'n_neighbors':list(range(20,31))}
neigh = KNeighborsClassifier()
clf = GridSearchCV(neigh, parameters)
clf.fit(X_train, y_train)
print(clf.best_params_)
pred = clf.predict(X_test)
print(f'Acuracy of the KNN{clf.best_params_}:{accuracy_score(y_test, pred)}')
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=29)
neigh.fit(X_train, y_train)
pred = neigh.predict(X_test)
print(f'Acuracy of the KNN29:{accuracy_score(y_test, pred)}')
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred)) # recall is better, but still poor results

In [ ]:
# Linear SVM
svc = LinearSVC(penalty = 'l1', dual = False, max_iter = 10000, C = 1)
svc.fit(X_train, y_train)
pred = svc.predict(X_test)
print(f'Acuracy of the SVM:{accuracy_score(y_test, pred)}')
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
# SVM
# I tested all these kernels: linear, poly, rbf, sigmoid and the best one is sigmoid with C = 0.7
svc = SVC(max_iter = -1, kernel = 'sigmoid', C = 0.7)
svc.fit(X_train, y_train)
pred = svc.predict(X_test)
print(f'Acuracy of the SVM:{accuracy_score(y_test, pred)}')
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred)) # best result

### Section 3: KNN, Linear SVM and SVM with LDA ###

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

lda = LDA(n_components=1)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

In [ ]:
# KNN
parameters = {'n_neighbors':list(range(20,31))}
neigh = KNeighborsClassifier()
clf = GridSearchCV(neigh, parameters)
clf.fit(X_train, y_train)
print(clf.best_params_)
pred = clf.predict(X_test)
print(f'Acuracy of the KNN{clf.best_params_}:{accuracy_score(y_test, pred)}')
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred)) # PCA better than LDA in KNN

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=24)
neigh.fit(X_train, y_train)
pred = neigh.predict(X_test)
print(f'Acuracy of the KNN24:{accuracy_score(y_test, pred)}')
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred)) # recall is better, but still poor results

In [ ]:
# Linear SVM
svc = LinearSVC(penalty = 'l1', dual = False, max_iter = 10000, C = 1)
svc.fit(X_train, y_train)
pred = svc.predict(X_test)
print(f'Acuracy of the SVM:{accuracy_score(y_test, pred)}')
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred)) # best result

In [ ]:
# SVM
# I tested all these kernels: linear, poly, rbf, sigmoid and the best one is rbf with C=0.5
svc = SVC(max_iter = -1, kernel = 'rbf', C=0.5)
svc.fit(X_train, y_train)
pred = svc.predict(X_test)
print(f'Acuracy of the SVM:{accuracy_score(y_test, pred)}')
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

### Section 4: Random Forest Implementation ###


In [ ]:
# Random forest implementation
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(classification_report(y_test, pred))


              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96     16756
         1.0       0.69      0.26      0.38      1587

    accuracy                           0.93     18343
   macro avg       0.81      0.62      0.67     18343
weighted avg       0.91      0.93      0.91     18343



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import pprint 
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 4)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 30, num = 5)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pp = pprint.PrettyPrinter(width=41, compact=True)
pp.pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 15, 20, 25, 30],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 800, 1400, 2000]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
pred = rf_random.predict(X_test)
print(rf_random.best_estimator_)
print(classification_report(y_test, pred))


### Section 5: Random Forest Implementation with LDA ###

In [ ]:
# Running LDA
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

lda = LDA(n_components=1)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 800, 1400, 2000]},
                   random_state=42, verbose=2)

In [ ]:
pred = rf_random.predict(X_test)
print(rf_random.best_estimator_)
print(classification_report(y_test, pred))

RandomForestClassifier(max_depth=10, min_samples_split=10, n_estimators=1400)
              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96     16756
         1.0       0.62      0.22      0.33      1587

    accuracy                           0.92     18343
   macro avg       0.78      0.61      0.64     18343
weighted avg       0.90      0.92      0.90     18343



### Section 6: Random Forest Implementation with PCA ###

In [ ]:
# Running PCA
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

from sklearn.decomposition import PCA
pca = PCA(n_components=0.7) # I tried 0.5 and 0.8, worser results
X_train = pca.fit_transform(X_train, y_train)
X_test = pca.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 800, 1400, 2000]},
                   random_state=42, verbose=2)

In [ ]:
pred = rf_random.predict(X_test)
print(rf_random.best_estimator_)
print(classification_report(y_test, pred))

RandomForestClassifier(max_depth=10, min_samples_split=10, n_estimators=1400)
              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96     16756
         1.0       0.67      0.25      0.36      1587

    accuracy                           0.92     18343
   macro avg       0.80      0.62      0.66     18343
weighted avg       0.91      0.92      0.91     18343




### Section 7: Naive Bayes, Logistic Regression and Decision Tree###

### a) Preprocessing to Encode Categorical Features ###

In [ ]:
# 1. Encode ethnicity, gender, icu_admit_source, icu_stay_type, icu_type, apache_2_bodysystem:
df = pd.concat([df,pd.get_dummies(df['ethnicity'], prefix='ethnic')],axis=1)
df = pd.concat([df,pd.get_dummies(df['gender'], prefix='gender')],axis=1)
df = pd.concat([df,pd.get_dummies(df['icu_admit_source'], prefix='admit_source')],axis=1)
df = pd.concat([df,pd.get_dummies(df['icu_stay_type'], prefix='stay_type')],axis=1)
df = pd.concat([df,pd.get_dummies(df['icu_type'], prefix='icu_type')],axis=1)
df = pd.concat([df,pd.get_dummies(df['apache_2_bodysystem'], prefix='ap2bodsys')],axis=1)

In [ ]:
# 2. Drop ethnicity, gender, icu_admit_source, icu_stay_type, icu_type, apache_2_bodysystem and apache_3j_bodysystem:
df.drop(['ethnicity'],axis=1, inplace=True)
df.drop(['gender'],axis=1, inplace=True)
df.drop(['icu_admit_source'],axis=1, inplace=True)
df.drop(['icu_stay_type'],axis=1, inplace=True)
df.drop(['icu_type'],axis=1, inplace=True)
df.drop(['apache_2_bodysystem'], axis=1, inplace=True)
df.drop(['apache_3j_bodysystem'], axis=1, inplace=True)

In [ ]:
df['life_or_death'] = df['hospital_death'].values
df.drop(['hospital_death'], axis=1, inplace=True)

### b) Splitting into Training and Test Sets ###

In [ ]:
# Select X and y values:

X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [ ]:
# Splitting dataset into training and testset

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


Part 1: Base Models (Naïve Bayes, Logistic and Decision Tree)

In [ ]:
##############################
########## PART 1 ############
##############################

##############################
########## Model 1: ##########

# Training Naive Bayes Model

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [ ]:
# Accuracy Score & Confusion Matrix for Naive Bayes Model 

from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[13997  2675]
 [  556  1115]]


0.8238565120209345

In [ ]:
# Classification Report for Naive Bayes Model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.84      0.90     16672
           1       0.29      0.67      0.41      1671

    accuracy                           0.82     18343
   macro avg       0.63      0.75      0.65     18343
weighted avg       0.90      0.82      0.85     18343



In [ ]:
##############################
########## Model 2: ##########

# Training Logistic Regression Model on Training Set

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0, max_iter = 300)
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=300, random_state=0)

In [ ]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

[[0 0]
 [1 1]
 [0 0]
 ...
 [0 0]
 [0 1]
 [0 0]]


In [ ]:
# Accuracy Score & Confusion Matrix for Logistic Regression 
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)

accuracy_score(y_test, y_pred)

[[16468   204]
 [ 1242   429]]


0.9211688382489233

In [ ]:
# Classification Report for Logistic Regression
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96     16672
           1       0.68      0.26      0.37      1671

    accuracy                           0.92     18343
   macro avg       0.80      0.62      0.67     18343
weighted avg       0.91      0.92      0.90     18343



In [ ]:
##############################
########## Model 3: ##########

# Training Decision Tree Model on Training Set

from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[15541  1131]
 [ 1083   588]]


0.879300005451671

In [ ]:
# Classification Report for Decision Tree
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93     16672
           1       0.34      0.35      0.35      1671

    accuracy                           0.88     18343
   macro avg       0.64      0.64      0.64     18343
weighted avg       0.88      0.88      0.88     18343




### Section 8: Naive Bayes, Logistic Regression and Decision Tree with PCA ###

In [ ]:
##############################
########## PART 2 ############
##############################

# 1. Select X and y values:

X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

# 2. Splitting dataset into training and testset:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# 3. Execute PCA:

from sklearn.decomposition import PCA
pca = PCA(n_components = 0.7)
X_train = pca.fit_transform(X_train, y_train)
X_test = pca.transform(X_test)


In [ ]:
##############################
########## Model 1: ##########

# Training Naive Bayes Model

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [ ]:
# Accuracy Score & Confusion Matrix for Naive Bayes Model 

from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[13997  2675]
 [  556  1115]]


0.8238565120209345

In [ ]:
# Classification Report for Naive Bayes Model

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.84      0.90     16672
           1       0.29      0.67      0.41      1671

    accuracy                           0.82     18343
   macro avg       0.63      0.75      0.65     18343
weighted avg       0.90      0.82      0.85     18343



In [ ]:
##############################
########## Model 2: ##########

# Training Logistic Model

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0, max_iter = 300)
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=300, random_state=0)

In [ ]:
# Predict Logistic Regression:

y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

[[0 0]
 [1 1]
 [0 0]
 ...
 [0 0]
 [0 1]
 [0 0]]


In [ ]:
# Accuracy Score & Confusion Matrix for Logistic Regression 

from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)

accuracy_score(y_test, y_pred)

[[16468   204]
 [ 1242   429]]


0.9211688382489233

In [ ]:
# Classification Report for Logistic Regression

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96     16672
           1       0.68      0.26      0.37      1671

    accuracy                           0.92     18343
   macro avg       0.80      0.62      0.67     18343
weighted avg       0.91      0.92      0.90     18343



In [ ]:
##############################
########## Model 3: ##########

# Training Decision Tree Model on Training Set

from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[15541  1131]
 [ 1083   588]]


0.879300005451671

In [ ]:
# Classification Report for Decision Tree
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93     16672
           1       0.34      0.35      0.35      1671

    accuracy                           0.88     18343
   macro avg       0.64      0.64      0.64     18343
weighted avg       0.88      0.88      0.88     18343



### Section 9: Naive Bayes, Logistic Regression and Decision Tree with LDA ###

In [ ]:
##############################
########## PART 3 ############
##############################

# 1. Select X and y values:

X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

# 2. Splitting dataset into training and testset:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# 3. Execute LDA:

lda = LDA(n_components=1)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

In [ ]:
##############################
########## Model 1: ##########

# Training Naive Bayes Model

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [ ]:
# Accuracy Score & Confusion Matrix for Naive Bayes Model 

from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[16252   420]
 [ 1078   593]]


0.9183339693616094

In [ ]:
# Classification Report for Naive Bayes Model

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.96     16672
           1       0.59      0.35      0.44      1671

    accuracy                           0.92     18343
   macro avg       0.76      0.66      0.70     18343
weighted avg       0.91      0.92      0.91     18343



In [ ]:
##############################
########## Model 2: ##########

# Training Logistic Model

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0, max_iter = 300)
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=300, random_state=0)

In [ ]:
# Predict Logistic Regression:

y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

[[0 0]
 [1 1]
 [0 0]
 ...
 [0 0]
 [1 1]
 [0 0]]


In [ ]:
# Accuracy Score & Confusion Matrix for Logistic Regression 

from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)

accuracy_score(y_test, y_pred)

[[16474   198]
 [ 1243   428]]


0.9214414217957804

In [ ]:
# Classification Report for Logistic Regression

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96     16672
           1       0.68      0.26      0.37      1671

    accuracy                           0.92     18343
   macro avg       0.81      0.62      0.67     18343
weighted avg       0.91      0.92      0.90     18343



In [ ]:
##############################
########## Model 3: ##########

# Training Decision Tree Model on Training Set

from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[15592  1080]
 [ 1164   507]]


0.8776645041705282

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[15592  1080]
 [ 1164   507]]


0.8776645041705282